This notebook will pull from the FPL API the weekly results for each player and store it in a dictionary

Update weekly

In [1]:
import pandas as pd
import numpy as np
import requests
import time

In [2]:
top500 = pd.read_csv('top500.csv')
top500.head()

,id_,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,...,threat,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
0,166,2,13,262,NaN,NaN,3,101668,1,-1,...,323.0,74,1126539,251903,539452,1921,1.1,8.0,Vardy,1
1,215,10,10,278,100.0,100.0,5,61366,1,-1,...,199.0,72,2729169,55005,1361423,7259,0.4,7.1,De Bruyne,0
2,192,3,8,185,100.0,100.0,3,110979,0,0,...,286.0,64,1717182,22566,701061,14271,0.5,5.4,Mané,0
3,214,1,7,223,NaN,NaN,4,103955,0,0,...,610.0,64,1447857,31525,1088554,10583,0.5,5.3,Sterling,3
4,210,3,6,258,NaN,NaN,2,37572,-1,1,...,392.0,62,1269546,6752,771337,120201,0.1,5.1,Agüero,0


In [4]:
histories = {}

ids = top500['id_']

for id_ in ids:
    url = 'https://fantasy.premierleague.com/api/element-summary/' + str(id_) + '/'
    r = requests.get(url)
    player_data = r.json()
    player_history = player_data['history']
    histories[id_] = player_history
    
    

In [5]:
history_dfs = {}

for key in histories:
    history_dfs[key] = pd.DataFrame(histories[key])
    history_dfs[key]['id_'] = key

In [6]:
type(history_dfs)

dict

In [7]:
top500histories = pd.concat(history_dfs.values(), ignore_index=True)
top500histories.head()

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,id_
0,0,0,5,1,4.9,166,5,0,0,1.6,...,0,4.0,2,0,0,0,90,True,0,166
1,0,0,-3,0,2.3,166,14,1,0,0.9,...,1,10.0,2,-82257,18334,100591,90,False,0,166
2,0,2,25,0,1.8,166,27,1,1,6.2,...,1,27.0,8,-78612,32101,110713,89,False,0,166
3,1,3,63,0,13.6,166,36,1,2,14.6,...,3,45.0,16,-7210,49194,56404,89,True,0,166
4,0,0,5,0,11.3,166,45,1,0,1.7,...,1,4.0,2,83507,136063,52556,89,False,0,166


In [8]:
top500histories.to_csv('top500histories.csv')